# Python Text Analysis: Word Embeddings

Thus far, we've focused on bag-of-word approaches to text analysis, where the text is represented as a vector of word frequencies. This generally works pretty well - we can do a decent job of supervised classification with this approach. However, word frequencies alone don't tell the whole picture. The ordering of words, for example, provides additional context that word frequencies don't capture. Furthermore, words can be used in a variety of ways, with different meanings that get lost in a word frequency representation.

An alternative formalization of text consists of representing the words (or bi-grams, phrases, etc.) as vectors. They're also called word embeddings, because we embed the word in a higher dimensional space. A word vector has no inherent meaning to humans - ultimately, it's just a bunch of floating point numbers. But word vectors are useful because they're a numerical representation of text that captures its semantic meaning, and can easily be used in downstream tasks, such as dictionary methods, classification, topic modeling etc. Furthermore, the vector representation can be used to perform semantic tasks, such as finding synonyms, testing analogies, and others. The big question, however, is: how do we create the word vector in the first place?

The answer is to pick the right task. Specifically, we're going to calculate the word vectors so that they can be successfully used in one of two tasks: predicting surrounding words, or predicting words within a context.

# The Word Embedding Model: `word2vec`

The word embedding model, generally referred to as `word2vec`, was developed by [Mikolov et al.](https://arxiv.org/abs/1310.4546) in 2013. The basic premise is to find vector representations of tokens that have semantic meaning. How do we go about learning a "good" vector representation from the data?

Mikolov et al. proposed two approaches: the **skip-gram (SG)** and the **continuous bag-of-words (CBOW)**. Both approaches are similar in that we use the vector representation of a token to try and predict what the nearby tokens are with a shallow neural network.

![word2vec](../images/word2vec.png)

In the continuous bag-of-words model, our goal is to predict a word $w(t)$, given the words that surround it - e.g., $w(t-2), w(t-1), w(t+1), w(t+2)$, etc. So, in an example text such as `I went to the store to get some apples`, we may try to use the word vectors for `I`, `went`, `to`, `the`, `to`, `get`, `some`, `apples` to predict the word `store`. This would correspond to a *window size* of 4: 4 words on either side of the target word.

In the skip-gram model, we construct a word vector that can be used to predict the words surrounding a specific word $w(t)$. This is the reverse of the continuous bag-of-words, and is a harder task, since we have to predict more from less information. In the above example, we'd aim to predict the remaining words in the sentence from the word vector for `store`. 

You can use either approaches to build a set of word embeddings. Mikolov et al. demonstrated that the skip-gram works pretty well in larger corpuses. Furthermore, it's easier to train the skip-gram efficiently, making it faster.

The mechanics of how the training is actually done revolves around a **shallow neural network**. An **objective function** is specified - a mathematical expression that quantifies how well we predicted a word - which allows the values of the word vectors to be optimized using **back propagation**. We won't go into these details for this workshop, but check out the Python Deep Learning workshop if you'd like to learn more about neural networks!

Let's jump into it!

# Installing `gensim`

In [1]:
import numpy as np
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

We'll be using a package called `gensim` to conduct our word embedding experiments. `gensim` is one of the major Python packages for natural language processing, largely aimed at using different kinds of embeddings.

If you don't have `gensim` installed, you can install it directly within this notebook:

In [2]:
# Run if you do not have gensim installed
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20430 sha256=3d9023e57dc2733a191ab9b0c6326ccc5b14969aedfe6461d3929ca9ae93de36
  Stored in directory: /Users/blueraspberry/Library/Caches/pip/wheels/01/02/ee/df0699282986903a384b69aab4413af9efd26b3612b5dccc9e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3513 sha256=a127cdc5407d7737a89394dee9cebd5eb1986b868d5ab9f1039f275518334754
  Stored in directory: /Users/blueraspberry/Library/Caches/pip/wheels/43/aa/48/5c66b931ff013ad19774081aa19656637af5c0cc33b5494b30
Successfully built fst-pso miniful


In [3]:
import gensim
import gensim.downloader as api

# Using Pre-trained Word Embeddings

The first thing we'll do is use a pre-trained word embedding. This means that we're downloading a word embedding model that has already been trained on a large corpus. Researchers have trained a variety of models in different contexts that are freely available on `gensim`. We can take a look at a few of them by looking in the `gensim` downloader:

In [4]:
gensim_models = list(api.info()['models'].keys())
print(gensim_models)

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


We are going to use the `word2vec-google-news-300` model: this is a word embedding model that is trained on Google News, where the embedding is 300 dimensions. Downloading this might take a while! The word embedding model is nearly 2 GB. 

In [5]:
wv = api.load('word2vec-google-news-300')

[--------------------------------------------------] 1.4% 23.4/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==------------------------------------------------] 5.5% 92.2/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[====----------------------------------------------] 9.8% 162.8/1662.8MB downloaded

...

[=============================================-----] 90.3% 1502.1/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[===============================================---] 95.2% 1583.0/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=================================================-] 100.0% 1662.3/1662.8MB downloaded


How many word vectors are available in this word embedding model? We can access the `index_to_key` member variable to find out:

In [6]:
n_words = len(wv.index_to_key)
print(f"Number of words: {n_words}")
print(wv.index_to_key[:20])

Number of words: 3000000
['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said', 'was', 'the', 'at', 'not', 'as', 'it', 'be', 'from', 'by', 'are']


The model is trained using a vocabulary of size 3 million! This is a huge model, which takes hours to train. This is why we used a pre-trained model - we likely don't have the resources to train this on our local machines.

Accessing the actual word vectors can be done by treating the word vector model as a dictionary. For example, let's take a look at the word vector for `"banana"`:

In [7]:
print(wv["banana"])
print(wv["banana"].size)

[-8.54492188e-02  4.71191406e-02 -6.93359375e-02  3.02734375e-01
 -1.87500000e-01 -3.19824219e-02  2.95410156e-02 -2.05078125e-01
 -9.03320312e-02  2.98828125e-01  2.10937500e-01 -6.88476562e-02
  4.76074219e-02  5.83496094e-02 -2.75390625e-01  2.42187500e-01
 -3.22265625e-01  4.73632812e-02 -1.44531250e-01  9.37500000e-02
  6.74438477e-03  2.17773438e-01  2.31445312e-01  1.87500000e-01
  8.49609375e-02 -5.39550781e-02 -2.65625000e-01  2.23388672e-02
  2.24609375e-01  4.12109375e-01 -2.30712891e-02 -1.67968750e-01
  1.01928711e-02  2.10937500e-01  1.14135742e-02  2.50000000e-01
  8.64257812e-02 -2.16796875e-01  8.93554688e-02  1.50390625e-01
 -2.03125000e-01 -2.30468750e-01  1.42578125e-01  1.34765625e-01
 -1.13769531e-01 -1.80664062e-01  3.58886719e-02 -1.69921875e-01
  1.55273438e-01  2.39257812e-01 -2.30468750e-01 -9.96093750e-02
  1.82617188e-01 -1.81640625e-01 -1.46484375e-01 -7.03125000e-02
  9.37500000e-02 -2.67578125e-01  2.17285156e-02 -1.45507812e-01
 -2.38037109e-02  1.53320

As promised, the word vector is 300-dimensional. Looking at the actual values of the vector is pretty uninformative - the values appear to be random floats. However, now that the word has been transformed into a vector, we can more easily perform computations on it that correspond to semantic operations. Let's take a look at a few examples.

## Word Similarity

A semantic question we can ask is  that are similar to "banana". How does word similarity look in vector operations? We'd expect similar words to have vectors that are closer to each other in vector space.

There are many metrics of vector similarity - one of the most useful ones is the [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). It has a range of 0 to 1, with orthogonal vectors have a cosine similarity of 0, and parallel vectors having a cosine similarity of 1. `gensim` provides a function that lets us find the most similar vectors to a queried vector - let's give it a shot! 

In [8]:
wv.most_similar('banana')

[('bananas', 0.752377450466156),
 ('pineapple', 0.6587537527084351),
 ('mango', 0.6365211009979248),
 ('pineapples', 0.6313878297805786),
 ('papaya', 0.6063666343688965),
 ('coconut', 0.6035483479499817),
 ('potato', 0.5747568607330322),
 ('melon', 0.5625776052474976),
 ('cashew', 0.562165379524231),
 ('tomato', 0.5590205788612366)]

The most similar vectors to "banana" are other fruits and foods! These are conceptual relationships that are reflected in the word embedding that we did not explicitly train in the model. Let's try another, more abstract word:

In [9]:
wv.most_similar('happy')

[('glad', 0.7408890724182129),
 ('pleased', 0.6632170677185059),
 ('ecstatic', 0.6626912355422974),
 ('overjoyed', 0.6599286794662476),
 ('thrilled', 0.6514049172401428),
 ('satisfied', 0.6437949538230896),
 ('proud', 0.636042058467865),
 ('delighted', 0.627237856388092),
 ('disappointed', 0.6269949674606323),
 ('excited', 0.6247665286064148)]

We see synonyms of "happy", and even an antonym ("disappointed"). 

## Challenge 1

Look up the `doesnt_match` function in `gensim`'s documentation. Use this function to identify which word doesn't match in the following group:

banana, apple, strawberry, happy

Then, try it on groups of words that you choose. Here are some suggestions:

1. A group of fruits, and a vegetable. Can it identify that the vegetable doesn't match?
2. A group of vehicles that travel by land, and a vehicle that travels by air (e.g., a plane or helicopter). Can it identify the vehicle that flies?
3. A group of scientists (e.g., biologist, physicist, chemist, etc.) and a person who does not study an empirical science (e.g., an artist). Can it identify the occupation that is not science based?

To be clear, `word2vec` does not learn the precise nature of the differences between these groups. However, the semantic differences correspond to similar words appearing near each other in large corpora.

In [13]:
help(gensim)

Help on package gensim:

NAME
    gensim

DESCRIPTION
    This package contains functionality to transform documents (strings) into vectors, and calculate
    similarities between documents.

PACKAGE CONTENTS
    _matutils
    corpora (package)
    downloader
    interfaces
    matutils
    models (package)
    nosy
    parsing (package)
    scripts (package)
    similarities (package)
    test (package)
    topic_coherence (package)
    utils

DATA
    logger = <Logger gensim (WARNING)>

VERSION
    4.3.0

FILE
    /Users/blueraspberry/anaconda3/lib/python3.10/site-packages/gensim/__init__.py




In [14]:
wv.doesnt_match(['banana', 'apple', 'strawberry', 'happy'])

'happy'

In [15]:
# A group of fruits, and a vegetable
wv.doesnt_match(['banana', 'apple', 'strawberry', 'lettuce'])

'lettuce'

In [17]:
# A group of vehicles that travel by land, and a vehicle that travels by air
wv.doesnt_match(['car', 'motorcycle', 'tank', 'glider'])

# Got it wrong! It picked tank instead of glider!

'tank'

In [18]:
# A group of scientists (e.g., biologist, physicist, chemist, etc.) and a person who does not study an empirical science (e.g., an artist). Can it identify the occupation that is not science based?
wv.doesnt_match(['biologist', 'physicist', 'chemist', 'dancer'])

'dancer'

## Word Analogies

One of the most famous usages of `word2vec` is via word analogies. For example:

`Paris : France :: Berlin : Germany`

Here, the analogy is between (Paris, France) and (Berlin, Germany), with "capital city" being the concept that connects them. We can abstract the "analogy" relationship to vector modeling. Let's pretend we're working with each of the vectors. Then, the analogy is

$\mathbf{v}_{\text{France}} - \mathbf{v}_{\text{Paris}} \approx \mathbf{v}_{\text{Germany}} - \mathbf{v}_{\text{Berlin}}.$

The vector difference here represents the notion of "capital city". Presumably, going from the Paris vector to the France vector (i.e., the vector difference) will be the same as going from the Berlin vector to the Germany vector, if that difference carries similar semantic meaning.

Let's test this directly. We'll do so by rewriting the above expression:

$\mathbf{v}_{\text{France}} - \mathbf{v}_{\text{Paris}} + \mathbf{v}_{\text{Berlin}} \approx \mathbf{v}_{\text{Germany}}.$

We'll calculate the difference between Paris and France, add on Germany, and find the closest vector to that quantity. Notice that, in all these operations, we set `norm=True`, and renormalize. That's because different vectors might be of different lengths, so the normalization puts everything on a common scale.

In [19]:
# Calculate "capital city" vector difference
difference = wv.get_vector('France', norm=True) - wv.get_vector('Paris', norm=True) 
# Add on Berlin
difference += wv.get_vector('Berlin', norm=True)
# Renormalize vector
difference /= np.linalg.norm(difference)

In [20]:
# What is the most similar vector?
wv.most_similar(difference)

[('Germany', 0.7901254892349243),
 ('Berlin', 0.6956422924995422),
 ('France', 0.6161972284317017),
 ('Austria', 0.6026812791824341),
 ('German', 0.6004960536956787),
 ('Germans', 0.5851002931594849),
 ('Poland', 0.5847076177597046),
 ('Hungary', 0.5271855592727661),
 ('BBC_Tristana_Moore', 0.5249711275100708),
 ('symbol_RSTI', 0.5245768427848816)]

Germany is the most similar! So, word analogies seem possible with `word2vec`.

Carrying out these operations can be done in one fell swoop with the `most_similar` function. Check the documentation for this function. What do the `positive` and `negative` arguments mean?

## Challenge 2

Carry out the following word analogies:

1. Mouse : Mice :: Goose : ?
2. Kangaroo : Joey :: Cat : ?
3. United States : Dollar :: Mexico : ?
4. Happy : Sad :: Up : ?
5. California : Sacramento :: Canada : ?
6. California : Sacramento :: Washington : ?

What about something more abstract, such as:

7. United States : hamburger :: Canada : ?

Some work well, and others don't work as well. Try to come up with your own analogies!

In [21]:
wv.most_similar(positive=['mice', 'goose'], negative=['mouse'])

[('geese', 0.63051438331604),
 ('Canada_geese', 0.5270898342132568),
 ('mallards', 0.5100248456001282),
 ('birds', 0.5052502155303955),
 ('wild_turkeys', 0.502317488193512),
 ('goslings', 0.4923616051673889),
 ('waterfowl', 0.490485817193985),
 ('pheasants', 0.4903114140033722),
 ('mallard', 0.48380815982818604),
 ('fawns', 0.4810999929904938)]

In [22]:
wv.most_similar(positive=['kangaroo', 'cat'], negative=['joey'])

[('dog', 0.5410016775131226),
 ('poodle', 0.5067529678344727),
 ('cats', 0.5043997764587402),
 ('pet', 0.4962843358516693),
 ('feline', 0.4906769096851349),
 ('rabbit', 0.4799746870994568),
 ('monkey', 0.4723745882511139),
 ('dogs', 0.46669694781303406),
 ('felines', 0.4661637544631958),
 ('rodent', 0.46047261357307434)]

In [23]:
wv.most_similar(positive=['Dollar', 'Mexico'], negative=['United_States'])

[('Peso', 0.5553824305534363),
 ('Greenback', 0.4611453115940094),
 ('Financial_DLLR', 0.4427323043346405),
 ('Romanian_Leu', 0.4391007125377655),
 ('Firmer_Against', 0.4282413721084595),
 ('Mexcio', 0.4204976260662079),
 ('Hungarian_Forint', 0.4182077944278717),
 ('Loonie', 0.41796350479125977),
 ('Peso_Opens', 0.4157997965812683),
 ('Colombian_Peso', 0.4136069118976593)]

In [24]:
wv.most_similar(positive=['sad', 'up'], negative=['happy'])

[('down', 0.47779443860054016),
 ('off', 0.37129518389701843),
 ('aside', 0.36451300978660583),
 ('in.', 0.35753732919692993),
 ('out', 0.35602906346321106),
 ('saddening', 0.354120671749115),
 ('apart', 0.3414230942726135),
 ('horrifying', 0.3372592031955719),
 ('Turkoman_Shiites', 0.33696192502975464),
 ('ups', 0.33589640259742737)]

In [25]:
wv.most_similar(positive=['Sacramento', 'Canada'], negative=['California'])

[('Ottawa', 0.6629177331924438),
 ('Saskatoon', 0.635101318359375),
 ('Toronto', 0.63128662109375),
 ('Winnipeg', 0.6305069923400879),
 ('Vancouver', 0.5973448157310486),
 ('Edmonton', 0.5950410962104797),
 ('Fredericton', 0.5898652076721191),
 ('Calgary', 0.5881612300872803),
 ('Thunder_Bay', 0.5864453315734863),
 ('Manitoba', 0.5829135179519653)]

In [26]:
wv.most_similar(positive=['Sacramento', 'Washington'], negative=['California'])

[('Spokane', 0.5503137111663818),
 ('Seattle', 0.5467826128005981),
 ('Washingon', 0.5098150968551636),
 ('Portland', 0.5031915307044983),
 ('Washinton', 0.4988557994365692),
 ('Wash_ington', 0.4968692362308502),
 ('Wizards_Etan_Thomas', 0.48660945892333984),
 ('Yakima', 0.4854264259338379),
 ('Bellevue', 0.47607871890068054),
 ('Tacoma', 0.47583305835723877)]

In [27]:
wv.most_similar(positive=['hamburger', 'Canada'], negative=['United_States'])

[('poutine', 0.6137834787368774),
 ('donair', 0.5990861654281616),
 ('burger', 0.588463544845581),
 ('hotdog', 0.5695340633392334),
 ('Swiss_Chalet', 0.5682101845741272),
 ('Timmies', 0.5621650218963623),
 ('burgers', 0.5535664558410645),
 ('hamburgers', 0.5418270826339722),
 ('perogies', 0.5386622548103333),
 ('peameal_bacon', 0.5319751501083374)]

In [32]:
wv.most_similar(positive=['work', 'rain'], negative=['vacation'])

[('heavy_rain', 0.5466160178184509),
 ('Rain', 0.5086942315101624),
 ('downpour', 0.5026025772094727),
 ('rains', 0.4748978912830353),
 ('downpours', 0.47341033816337585),
 ('torrential_rain', 0.47282513976097107),
 ('drizzle', 0.4708229899406433),
 ('heavy_downpour', 0.4535405933856964),
 ('wet_weather', 0.45113813877105713),
 ('torrential_downpour', 0.45040756464004517)]

# Creating Custom Word Embeddings

In the previous example, we used a *pretrained* word embedding. That is, the word embedding was already trained using a very large corpus from Google News. What about when we want to train our own word embeddings from a new corpus?

We can do that using `gensim` as well. However, if the corpus is large, training becomes very computationally taxing. So, we'll try training our own word embeddings, but on a much smaller corpus. Specifically, we'll return to one you should recognize: the airline tweets corpus!

Let's go ahead and get set up by importing the dataset and preprocessing, as we did in Part 2.

In [33]:
tweets_path = '../data/airline_tweets.csv'
tweets = pd.read_csv(tweets_path, sep=',')

In [34]:
def preprocess(text):
    """Preprocesses a string."""
    # Lowercase
    text = text.lower()
    # Replace URLs
    url_pattern = r'https?:\/\/.*[\r\n]*'
    url_repl = ' URL '
    text = re.sub(url_pattern, url_repl, text)
    # Replace digits
    digit_pattern = '\d+'
    digit_repl = ' DIGIT '
    text = re.sub(digit_pattern, digit_repl, text)
    # Replace hashtags
    hashtag_pattern = r'(?:^|\s)[＃#]{1}(\w+)'
    hashtag_repl = ' HASHTAG '
    text = re.sub(hashtag_pattern, hashtag_repl, text)
    # Replace users
    user_pattern = r'@(\w+)'
    user_repl = ' USER '
    text = re.sub(user_pattern, user_repl, text)
    # Remove blank spaces
    blankspace_pattern = r'\s+'
    blankspace_repl = ' '
    text = re.sub(blankspace_pattern, blankspace_repl, text).strip()
    return text

In [35]:
tweets['text_processed'] = tweets['text'].apply(lambda x: preprocess(x))
tweets['text_processed'].head()

0                                 USER what USER said.
1    USER plus you've added commercials to the expe...
2    USER i didn't today... must mean i need to tak...
3    USER it's really aggressive to blast obnoxious...
4        USER and it's a really big bad thing about it
Name: text_processed, dtype: object

To create our own model, we need to import the `Word2Vec` module from `gensim`:

In [36]:
from gensim.models import Word2Vec

You can check out the documentation for this module [here](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec). The main input to `Word2Vec` is a `sentences` argument, which consists of a list of lists: the outer list enumerates the documents, and the inner list enumerates the tokens within in each list. So, we need to run a word tokenizer on each of the tweets. Let's use `nltk`'s word tokenizer:

In [37]:
from nltk.tokenize import word_tokenize

In [38]:
sentences = [word_tokenize(tweet) for tweet in tweets['text_processed']]

In [39]:
sentences[0]

['USER', 'what', 'USER', 'said', '.']

Now, we train the model. We are going to use CBOW to train the model since it's better suited for smaller datasets. Take note of what other arguments we set:

In [40]:
model = Word2Vec(
    sentences=sentences,
    vector_size=30,
    window=5,
    min_count=1,
    sg=0)

The model is now trained! Let's take a look at some word vectors. We can access them using the `wv` attribute:

In [41]:
len(model.wv)

11416

In [42]:
model.wv['worst']

array([ 2.2056377 , -1.9333711 ,  0.5149641 , -0.19023997,  1.3556246 ,
        0.14056772,  1.1613837 ,  0.92067295, -0.92699295,  0.02519775,
        0.5270743 , -0.18257314,  0.91322386, -1.1264545 ,  0.49621043,
       -0.5633961 ,  1.554221  ,  0.6007466 ,  0.8047933 ,  1.9140939 ,
       -0.40510353, -1.577057  ,  1.3557762 ,  0.6834252 ,  0.9947211 ,
        1.6152983 ,  0.10821747,  0.9800965 ,  0.33582798, -2.3646502 ],
      dtype=float32)

Let's try running a `most_similar` query to see what we end up with:

In [43]:
model.wv.most_similar('worst')

[('experience', 0.9590411186218262),
 ('horrible', 0.9436632394790649),
 ('ever', 0.9369516968727112),
 ('dealt', 0.9329656362533569),
 ('terrible', 0.9213968515396118),
 ('disappointed', 0.9093981981277466),
 ('awful', 0.8859463334083557),
 ('experienced', 0.8848305940628052),
 ('experiences', 0.8834440112113953),
 ('officially', 0.8824867010116577)]

In [44]:
model.wv.most_similar('great')

[('awesome', 0.9499430060386658),
 ('horrible', 0.9134411215782166),
 ('amazing', 0.9132348895072937),
 ('excellent', 0.9080108404159546),
 ('truly', 0.9033851027488708),
 ('terrible', 0.893530011177063),
 ('job', 0.8933104872703552),
 ('incredible', 0.8909650444984436),
 ('sucks', 0.8887729644775391),
 ('😊', 0.8882960677146912)]

In [45]:
model.wv.most_similar('united')

[('sw', 0.9786347150802612),
 ('water', 0.972813069820404),
 ('errors', 0.9717756509780884),
 ('virgin', 0.971594512462616),
 ('fraud', 0.9712207317352295),
 ('extremely', 0.9705446362495422),
 ('stopped', 0.9695917367935181),
 ('dept', 0.9691978693008423),
 ('zero', 0.9691344499588013),
 ('fails', 0.968288242816925)]

The `word2vec` model learned these relationships from the roughly 11,000 tweets in the corpus. These relationships look similar to some in the Google News word embeddings, have some differences that stem from the particular nature of the corpus, and the smaller number of documents.

## Challenge 3

Try experimenting with different numbers of vector sizes, window sizes, and other parameters available in the `Word2Vec` module. Additionally, try training using skip-grams rather than CBOW.

In [48]:
tweets_path = '../data/airline_tweets.csv'
tweets = pd.read_csv(tweets_path, sep=',')

In [49]:
def preprocess(text):
    """Preprocesses a string."""
    # Lowercase
    text = text.lower()
    # Replace URLs
    url_pattern = r'https?:\/\/.*[\r\n]*'
    url_repl = ' URL '
    text = re.sub(url_pattern, url_repl, text)
    # Replace digits
    digit_pattern = '\d+'
    digit_repl = ' DIGIT '
    text = re.sub(digit_pattern, digit_repl, text)
    # Replace hashtags
    hashtag_pattern = r'(?:^|\s)[＃#]{1}(\w+)'
    hashtag_repl = ' HASHTAG '
    text = re.sub(hashtag_pattern, hashtag_repl, text)
    # Replace users
    user_pattern = r'@(\w+)'
    user_repl = ' USER '
    text = re.sub(user_pattern, user_repl, text)
    # Remove blank spaces
    blankspace_pattern = r'\s+'
    blankspace_repl = ' '
    text = re.sub(blankspace_pattern, blankspace_repl, text).strip()
    return text

In [50]:
tweets['text_processed'] = tweets['text'].apply(lambda x: preprocess(x))
tweets['text_processed'].head()

0                                 USER what USER said.
1    USER plus you've added commercials to the expe...
2    USER i didn't today... must mean i need to tak...
3    USER it's really aggressive to blast obnoxious...
4        USER and it's a really big bad thing about it
Name: text_processed, dtype: object

In [51]:
sentences = [word_tokenize(tweet) for tweet in tweets['text_processed']]

In [52]:
model = Word2Vec(
    sentences=sentences,
    vector_size=50,
    window=5,
    min_count=2,
    sg=1)

In [55]:
model.wv.most_similar('sunshine')

[('albany', 0.9793749451637268),
 ('pressure', 0.9783806800842285),
 ('inches', 0.9754108786582947),
 ('oscar', 0.974621057510376),
 ('yourself', 0.972838819026947),
 ('vaca', 0.9727298021316528),
 ('yummy', 0.9720657467842102),
 ('calendar', 0.9719467759132385),
 ('troubadour', 0.9719133973121643),
 ('april', 0.9712861180305481)]

In [57]:
model.wv.most_similar('hotel')

[('room', 0.8966322541236877),
 ('food', 0.8506208062171936),
 ('voucher', 0.8441104292869568),
 ('vouchers', 0.8221388459205627),
 ('price', 0.8203084468841553),
 ('cost', 0.8143076300621033),
 ('baby', 0.8126083612442017),
 ('group', 0.8063298463821411),
 ('assigned', 0.8058637976646423),
 ('stay', 0.805268406867981)]

# Classifying with Trained Embeddings

In the previous module, we used the airline tweets dataset to perform sentiment classification: we tried to classify the sentiment of a text given the bag-of-words representation. Can we do something similar with a word embedding representation?

In the word embedding representation, we have an $N$-dimensional vector for each word in a tweet. How can we come up with a representation for the entire tweet?

The simplest approach we could take is to simply average the vectors together to come up with a "tweet representation". Let's see how this works for predicting sentiment classification.

First, we need to subset the dataset into the tweets which only have positive or negative sentiment:

In [58]:
tweets_binary = tweets[tweets['airline_sentiment'] != 'neutral']
y = tweets_binary['airline_sentiment']
print(y.value_counts(normalize=True))

negative    0.795252
positive    0.204748
Name: airline_sentiment, dtype: float64


Now, we need to compute the feature matrix. We will query the word vector in each tweet, and come up with an average for the sample:

In [61]:
vector_size = 30
X = np.zeros((len(y), vector_size))

# Enumerate over tweets
for idx, tweet in enumerate(tweets_binary['text_processed']):
    # Tokenize the current tweet
    tokens = word_tokenize(tweet)
    n_tokens = len(tokens)
    # Enumerate over tokens, obtaining word vectors
    for token in tokens:
        X[idx] += model.wv.get_vector(token)
    # Take the average
    X[idx] /= n_tokens

ValueError: operands could not be broadcast together with shapes (30,) (50,) (30,) 

As before, we'll proceed with splitting the data into train/test examples. We'll bring back the logistic fitter function from before, with some small changes.

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [63]:
def fit_logistic_regression(X, y):
    """Fits a logistic regression model to provided data."""
    model = LogisticRegressionCV(
        Cs=10,
        penalty='l2',
        max_iter=1000,
        cv=5,
        refit=True).fit(X, y)
    return model

We then run the fit, and evaluate it!

In [64]:
# Fit the logistic regression model
fitter = fit_logistic_regression(X_train, y_train)

In [65]:
print(f"Training accuracy: {fitter.score(X_train, y_train)}")
print(f"Test accuracy: {fitter.score(X_test, y_test)}")

Training accuracy: 0.7940857885615251
Test accuracy: 0.7999133824166306


While this performance is pretty good, it's definitely not as good as the bag-of-words representation we used in the previous module. There are few reasons this might be the case:

1. We used a word embedding on a relatively small corpus. A word embedding obtained from a very large corpus would perform better. The tricky part in doing this is that our smaller corpus may have some niche tokens that are not in the larger model, so we'd have to work around that.
2. We simply averaged word embeddings across tokens. When doing this, we lose meaning in the ordering of words. Other methods, such as `doc2vec`, have been proposed to address these concerns.
3. Word embeddings might be an overly complicated approach for the task at hand. In a tweet aimed at an airline, a person needs to convey their sentiment in only 140 characters. So they are more likely to use relatively simple words that easily convey sentiment, making a bag-of-words a natural approach.

It's important to note that we also lose out on the interpretability of the logistic regression model, because the actual dimensions of each word vector do not themselves have any meaning. 

Moral of the story: word embeddings are great, but always start with the simpler model! This is a good way to baseline other approaches, and it might actually work pretty well!

## Challenge 4

Write a function that performs the pipeline of building a `word2vec` model and constructing a design matrix. Use this function to try and see if you can change the performance of the model with other parameters (vector sizes, window sizes, etc.).

In [66]:
tweets_binary = tweets[tweets['airline_sentiment'] != 'neutral']
y = tweets_binary['airline_sentiment']

In [67]:
def featurizer(documents, to_train, vector_size=50, window=6, sg=0):
    """Computes a feature matrix from a document corpus."""
    sentences = [word_tokenize(doc) for doc in documents]
    # Train word2vec
    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window,
        min_count=1,
        sg=sg)
    
    X = np.zeros((len(to_train), vector_size))
    # Enumerate over tweets
    for idx, doc in enumerate(to_train):
        # Tokenize the current tweet
        tokens = word_tokenize(doc)
        n_tokens = len(tokens)
        # Enumerate over tokens, obtaining word vectors
        for token in tokens:
            X[idx] += model.wv.get_vector(token)
        # Take the average
        X[idx] /= n_tokens
    return X

In [68]:
X = featurizer(tweets['text_processed'], tweets_binary['text_processed'], vector_size=80, window=6)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [70]:
def fit_logistic_regression(X, y):
    """Fits a logistic regression model to provided data."""
    model = LogisticRegressionCV(
        Cs=5,
        penalty='l2',
        max_iter=1000,
        tol=1e-2,
        cv=3,
        refit=True).fit(X, y)
    return model

In [71]:
# Fit the logistic regression model
fitter = fit_logistic_regression(X_train, y_train)

/Users/blueraspberry/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/blueraspberry/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

In [72]:
print(f"Training accuracy: {fitter.score(X_train, y_train)}")
print(f"Test accuracy: {fitter.score(X_test, y_test)}")

Training accuracy: 0.8960138648180243
Test accuracy: 0.884365526201819
